In [ ]:
import reproducibility

In [ ]:
# both can change
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import json
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K
from keras.utils.vis_utils import plot_model
from scipy.interpolate import LinearNDInterpolator, interpn
from scipy.optimize import root
import pickle, os
from utils import hash2
%matplotlib inline

# Local modules for handling data and running moisture models
import data_funcs as datf
from data_funcs import format_raws, retrieve_raws, format_precip, fixnan
from data_funcs import raws_data, synthetic_data, plot_data, check_data, mse_data, to_json, from_json
import moisture_models as mod
from moisture_rnn import run_case, run_rnn, create_RNN_2, staircase, create_rnn_data, train_rnn, rnn_predict


In [ ]:
# Change directory for data read/write
os.chdir('data')

In [ ]:
# read test datasets
# case_data = from_json('rnn_orig.json')
filename = 'testing_dict.pickle'
with open(filename, 'rb') as handle:
    test_dict = pickle.load(handle)
    for case in test_dict:
        test_dict[case]['case'] = case
        test_dict[case]['filename'] = filename


In [ ]:
cases = ['case11']    # choose a test dataset
#cases = test_dict.keys() # run all cases

In [ ]:
for case in cases:
    print('\n***** ',case,' *****\n')
    case_data = test_dict[case]
    run_case(case_data,verbose=False,title2='',scale=0)
